# Tomography reconstruction of PUP_FF_SOFC_hires3_S2

The tomography data set PUP_FF_SOFC_hires3_S2 is used in this study.  __Four__ topics are discussed in this notebook:

* Using SVD based method to remove noises in the projection images (after background normalization)
* Using image processing method adaptive histogram equalization ([CLAHE](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization)) to enhance the details in reconstruction results
* Iterative method to compensate the horizontal sample jittering (locally) and drifting (globally)
    * standard phase correlation based rotation center locator
    * iterative global adjustment (IGA)
    * iterative pairwise adjustment (IPA)
* Various atttempt to compensate for the vertical sample jittering (locally) and drifting (globally)


In [2]:
from IPython.display import IFrame

## Evaluation of SVD based projection enhancement for tomography reconstruction

Singular value decomposition is a powerful tool that is commonly used in the noise reduction for realtime image stream, lossy images compression and feature detection. 
In this section, the application of SVD based image enhancement for tomography reconsutrction is investigated.

> Simple example demonstrating eigen feature (eigenimg)

![eigDemo](imgs/eignimg_2628.gif "eig_400")

>NOTE:  
Due to large amount of GIFs used in this notebook, it might __take a few minutes__ (depending on the network) for Binder to load all of them the browser.

### SVD enhanced projections with various n_eig

Left= original image , middle= eigen space, right= reconstructed from reduced eigen space (SVD method)

Standard View


| `n_eig = 400`   | `n_eig = 300`    |
| -----------   | -------------  |
| ![eigMax400](imgs/img_eigMax400.gif "eig_400") | ![eigMax300](imgs/img_eigMax300.gif "eig_300") | 

| `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------:|
|![eigMax160](imgs/img_eigMax160.gif "eig_160") | ![eigMax080](imgs/img_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20` |
| :-----:|:-----------: |
| ![eigMax040](imgs/img_eigMax040.gif "eig_40") | ![eigMax20]( imgs/img_eigMax020.gif "eig_20") | 

| `n_eig = 10`    | `n_eig = 5`  |
| :-------------:| :-----:|
|![eigMax010](imgs/img_eigMax010.gif "eig_10") | ![eigMax005](imgs/img_eigMax005.gif "eig_5") |

Compact View

| `n_eig = 400`   | `n_eig = 300`    | `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------: | :-----------: |:-------------:|
| ![eigMax400](imgs/img_eigMax400.gif "eig_400") | ![eigMax300](imgs/img_eigMax300.gif "eig_300") | ![eigMax160](imgs/img_eigMax160.gif "eig_160") | ![eigMax080](imgs/img_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20`    | `n_eig = 10`    | `n_eig = 5`  |
| :-----:|:-----------: |:-------------:| :-----:|
| ![eigMax040](imgs/img_eigMax040.gif "eig_40") | ![eigMax20]( imgs/img_eigMax020.gif "eig_20") | ![eigMax010](imgs/img_eigMax010.gif "eig_10") | ![eigMax005](imgs/img_eigMax005.gif "eig_5") |

The results above indicate that the first a few eigen features (~20) are sufficient in capturing most of the information about the sample whereas the remaining eigen features are most likely related to the various noise in the final images.
Therefore, it is possible to use the first 20 eigen features to extract the main features (sample) of the projection images, effectively separating the sample from the various shawdowy background noises.

### Tomography reconstruction using SVD enhanced images

Following the standard procedure, the rotation center of this data set can be found through phase correlation of the 180 degree pairs. 
However, the significant horizontal jittering and drifing of the sample makes it difficult of acquire clear reconstruction results.
Therefore, iterative global adjustment (IGA), a horizontal drift adjustment method, is used to __horizontally centering the sample__ from all __3601__ ($-180^\text{o} \to 180^\text{o}$, $\delta\omega=0.1^\text{o}$) images.

> The detailed analysis of IGA will be covered in the next section

With the enhanced and centered projection images, the tomograhy reconstruction can be eaisly done using existing toolkit (tomopy) with the command below

```python
recon = tomopy.recon(projs, thetas, 
                     center=rot_center, 
                     algorithm=recon_config['algorithm'],
                     filter_name=recon_config['filter'],
                    )
```
where the configuration of the reconsutrction is 
```python
recon_config = {'algorithm': 'gridrec',
                'filter'   : 'hann',
               }
```

__Tomo results using _enhanced_ and _centered_ projection images__

Standard View

| `n_eig = 400`   | `n_eig = 300`    | 
| :-----------: |:-------------: | 
| ![eigMax400](imgs/recon_conNeg_clipped_centered_eigMax400.gif "eig_400") | ![eigMax300](imgs/recon_conNeg_clipped_centered_eigMax300.gif "eig_300") | 

| `n_eig = 160`   | `n_eig = 80`   |
| :-----------:   |:-------------: |
| ![eigMax160](imgs/recon_conNeg_clipped_centered_eigMax160.gif "eig_160") | ![eigMax080](imgs/recon_conNeg_clipped_centered_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20`    | 
| :-----:|:-----------: |
| ![eigMax040](imgs/recon_conNeg_clipped_centered_eigMax040.gif "eig_40") | ![eigMax20](imgs/recon_conNeg_clipped_centered_eigMax020.gif "eig_20") | 

| `n_eig = 10`    | `n_eig = 5`  |
| :-------------:| :-----:|
| ![eigMax010](imgs/recon_conNeg_clipped_centered_eigMax010.gif "eig_10") | ![eigMax005](imgs/recon_conNeg_clipped_centered_eigMax005.gif "eig_5") |

Compact View

| `n_eig = 400`   | `n_eig = 300`    | `n_eig = 160`   | `n_eig = 80`    | 
| :-----------: |:-------------: | :-----------: |:-------------:|
| ![eigMax400](imgs/recon_conNeg_clipped_centered_eigMax400.gif "eig_400") | ![eigMax300](imgs/recon_conNeg_clipped_centered_eigMax300.gif "eig_300") | ![eigMax160](imgs/recon_conNeg_clipped_centered_eigMax160.gif "eig_160") | ![eigMax080](imgs/recon_conNeg_clipped_centered_eigMax080.gif "eig_80") |

| `n_eig = 40`  | `n_eig = 20`    | `n_eig = 10`    | `n_eig = 5`  |
| :-----:|:-----------: |:-------------:| :-----:|
| ![eigMax040](imgs/recon_conNeg_clipped_centered_eigMax040.gif "eig_40") | ![eigMax20](imgs/recon_conNeg_clipped_centered_eigMax020.gif "eig_20") | ![eigMax010](imgs/recon_conNeg_clipped_centered_eigMax010.gif "eig_10") | ![eigMax005](imgs/recon_conNeg_clipped_centered_eigMax005.gif "eig_5") |

The results above indicate that:

* The __more eigen features (vectors)__ used in the SVD enhancement, __the sharper__ the images. However, this effect is not linearly (__non-linearity__) depending no n_eig as the increase in sharpness plateaued quickly when n_eig passed 20.
* The artifact of rings, which is a known issues in Tomopy (see the ripple features in each GIF image), are more prominent with larger n_eig.  In the extreme cases where the reconstruction is only done with the first 5 eigen features (vectors), the ring artifacts disappeared completely.
    * it is also worth pointing out that the reconstruction quality at n_eig=5 is really low as the whole images are somewhat blurred.
    * the preojction images reconstructed from the first 5 eigen vectors looks very different from the original images (see previous sections), but the reconstruction results are roughly the same as the other.
* A significant portion (~10%) of the pixels in the reconstruction images have negative values.
    * these negative pixels are natural output of the tomograhy reconstruction, therefore they should not be clipped off by force
    * it is possible to shift and rescale the dynamic range of each image, however, this process often leads to large brightness variation along $\omega$.
* The CDF distribution remains very steady, regardless of the number of eigen features used for reconstruction.

Overall, the SVD based denoising method proves to be efficient in removing the unwanted background noises in the normalized images.  However, there are also several undesirable outcomes that comes with this method. 
The pros and cons of the proposed SVD based projection enhancement for tomography reconstruction is summarized in the table below.

| pros | cons | 
| :---: | :---: |
| efficient in removing unwanted noises in the normilzed images | fine details (pixel level) might lost in the process |
| easy to implement | the computation cost (mostly memory) increase rapidly with increasing image sizes |
| capable of capturing most features in the final reconstruction with merely 20 eigen features | the reconstruction results might look slighly blurred |

> NOTE:
The SVD based enhancement also has very little effect for the sample jittering/drifting adjustment, the details of which are covered in the next section.

## Enhance reconstruction results with adaptive histogram equilization

The tomography reconstruction results from high energy xray diffraction often has low contrast, making it difficult to spot the important features through visual inspection.
To overcome this issue, adaptive histogram equilization ([CLAHE](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization)) is used to bring out the details in the reconstruction results.
An __interactive__ example is provided in the cell below.

In [17]:
IFrame(src='docs/demoCLAHE.html', width=550, height=500)

>NOTE:
several features about CLAHE need to be pointed out here:

* CLAHE does not add _new_ features, it only amplify features based on local histogram
* CLAHE does not remove artifacts.
    * It might actually amplify the artifacts
* CLAHE can be computational expensive
    * The CLAHE performed here is using the CLAHE function provided by ImageJ

## Compensate for horizontal sample jittering/drifting

As mentioned in previous section, significant amount of sample jittering (locally) and drifting (globally) were found along the horizontal axis. 
In other words, the rotation center for each image are not necessearily correlated anymore.
Therefore, it is necesary to properly align samples from different images ($\omega$) such that a universal rotation center can be defined for the reconstruction process. 
To this end, two different method are proposed here:

* iterative global adjustment (IGA)
    * calcuate the rotation center ($y_{rc}^\omega$) of each pair image ($\Delta\omega = 180^\text{o}$)
    * find the average rotatio center $\bar{y}_{rc}$ 
    * move the rotation center of each pair to the center column using $\bar{y}_{rc}$
    * calculate new average rotatio center $y_{rc}'$
    * repeat until $y_{rc}$ converges
* iterative pairwise adjustment (IPA)
    * calcuate the rotation center ($y_{rc}^\omega$) of each pair image ($\Delta\omega = 180^\text{o}$)
    * move rotation center of each image to the center column using $y_{rc}^\omega$
    * calculate average rotatio center $y_{rc}'$
    * repeat until $y_{rc}$ converges
   

### Profiling the horizontal jittering

Using phase correlation on image pairs that are $180^\text{o}$ away along $\omega$-axis, individual rotation centers $y_{rc}^\omega$ can be located for all images, the distribution of which can be used to evaluate the horizontal misalignment for the projections.

In [27]:
IFrame(src='imgs/rotcnt_stats.pdf', width=850, height=500)

The distribution above shows that the raw data (uncorrected) has a really large variance in the rotation center (~20 pixels).
The proposed IGA method can bring the average rotation center (mean of $y_{rc}^\omega$) down to the image column center (250).
However, this has little effect of the variance of $y_{rc}^\omega$.
Therefore the rotations centers in the projections remain scattered, but around a different/new average value.

On the other hand, the IGA method can bring almost all the rotation centers to 250, except for six $\omega$s (3 pairs).
Further investigation reveals that these three _uncorrectable_ pairs contains at least one corrupted images where the sample is nowhere to be found within the image, which explains why phase correlation returns unrealistically large values back. 
Since the foundation of phase correlation is the $180^\text{o}$ pairing, a total number of six image ($\omega$s) need to be exculded from the reconstruction process.
